In [1]:
!pip install confluent-kafka

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 1.6 MB/s eta 0:00:0000:0100:01


# Create topic 

In [2]:
from confluent_kafka.admin import AdminClient, NewTopic


admin_client = AdminClient({
    "bootstrap.servers": "kafka:9092"
})

topic_list = []
topic_list.append(NewTopic("postgres.public.dbz_test", 1, 1))
admin_client.create_topics(topic_list)

{'postgres.public.dbz_test': <Future at 0x7fbefeb557f0 state=running>}

# Postgress connector 

In [3]:
import requests
import json

data = {
  "name": "postgres-connector",  
  "config": {
    "connector.class": "io.debezium.connector.postgresql.PostgresConnector", 
    "database.hostname": "postgres", 
    "database.port": "5432", 
    "database.user": "postgres", 
    "database.password": "postgres", 
    "database.dbname" : "postgres", 
    "database.server.name": "postgres",
    "slot.name": "debezium",
    "plugin.name": "pgoutput",
    "table.include.list": "public.dbz_test"
  }
}

headers = {
    "Content-Type": "application/json"
}

url="http://connect:8083/connectors"

resp = requests.post(url, headers=headers, data=json.dumps(data))

resp.text

'{"name":"postgres-connector","config":{"connector.class":"io.debezium.connector.postgresql.PostgresConnector","database.hostname":"postgres","database.port":"5432","database.user":"postgres","database.password":"postgres","database.dbname":"postgres","database.server.name":"postgres","slot.name":"debezium","plugin.name":"pgoutput","table.include.list":"public.dbz_test","name":"postgres-connector"},"tasks":[],"type":"source"}'

# Iceberg sink

In [4]:
import requests
import json

data = {
  "name": "iceberg-sink",
  "config": {
    "connector.class": "com.getindata.kafka.connect.iceberg.sink.IcebergSink",
    "topics": "postgres.public.dbz_test",
	
    "upsert": True,
    "upsert.keep-deletes": True,
    
    "table.auto-create": True,
    "table.write-format": "parquet",
    "table.namespace": "mytable_dbz",
    "table.prefix": "debeziumcdc_",
    
    
    
    #AWS (OK)
    #"iceberg.warehouse": "s3a://qooba-streaminglabs-eu-west-1/dbz_iceberg/gl_test",
    #"iceberg.fs.defaultFS": "s3a://qooba-streaminglabs-eu-west-1/dbz_iceberg/gl_test", 
    #"iceberg.catalog.type": "hadoop", 
    #"iceberg.catalog-name": "mycatalog", 
    #"iceberg.catalog-impl": "org.apache.iceberg.hadoop.HadoopCatalog",
    #  
    #"iceberg.com.amazonaws.services.s3.enableV4": True,
    #"iceberg.com.amazonaws.services.s3a.enableV4": True,
    #"iceberg.fs.s3a.aws.credentials.provider": "com.amazonaws.auth.DefaultAWSCredentialsProviderChain",
    #"iceberg.fs.s3a.path.style.access": True,
    #"iceberg.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
    #  
    #"iceberg.fs.s3a.access.key": "***",
    #"iceberg.fs.s3a.secret.key": "***",
      
    ###"iceberg.catalog-impl": "org.apache.iceberg.aws.glue.GlueCatalog",
    ###"iceberg.warehouse": "s3a://mybucket/dbz_iceberg/gl_test",
    ###"iceberg.fs.defaultFS": "s3a://mybucket/dbz_iceberg/gl_test",  
      
    #MINIO
    "iceberg.warehouse": "s3a://mybucket",
    "iceberg.fs.defaultFS": "s3a://mybucket", 
    #"iceberg.type": "hadoop", 
    "iceberg.catalog-name": "mycatalog", 
    "iceberg.catalog-impl": "org.apache.iceberg.hadoop.HadoopCatalog", 
      
    #"iceberg.com.amazonaws.services.s3.enableV4": True,
    #"iceberg.com.amazonaws.services.s3a.enableV4": True,
    #"iceberg.fs.s3a.aws.credentials.provider": "com.amazonaws.auth.DefaultAWSCredentialsProviderChain",
    "iceberg.fs.s3a.path.style.access": True,
    "iceberg.fs.s3a.endpoint": "http://minio:9000",
    "iceberg.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
    "iceberg.fs.s3a.access.key": "minioadmin",
    "iceberg.fs.s3a.secret.key": "minioadmin",
  }
}

headers = {
    "Content-Type": "application/json"
}

url="http://connect:8083/connectors"

resp = requests.post(url, headers=headers, data=json.dumps(data))

resp.text

'{"name":"iceberg-sink","config":{"connector.class":"com.getindata.kafka.connect.iceberg.sink.IcebergSink","topics":"postgres.public.dbz_test","upsert":"true","upsert.keep-deletes":"true","table.auto-create":"true","table.write-format":"parquet","table.namespace":"mytable_dbz","table.prefix":"debeziumcdc_","iceberg.warehouse":"s3a://mybucket","iceberg.fs.defaultFS":"s3a://mybucket","iceberg.catalog-name":"mycatalog","iceberg.catalog-impl":"org.apache.iceberg.hadoop.HadoopCatalog","iceberg.fs.s3a.path.style.access":"true","iceberg.fs.s3a.endpoint":"http://minio:9000","iceberg.fs.s3a.impl":"org.apache.hadoop.fs.s3a.S3AFileSystem","iceberg.fs.s3a.access.key":"minioadmin","iceberg.fs.s3a.secret.key":"minioadmin","name":"iceberg-sink"},"tasks":[],"type":"sink"}'

In [76]:
import requests 

resp = requests.delete("http://connect:8083/connectors/iceberg-sink")
resp.text

''

In [9]:
from confluent_kafka import Producer


p = Producer({'bootstrap.servers': 'kafka:9092'})

def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))

some_data_source = ['test']
        
for data in some_data_source:
    # Trigger any available delivery report callbacks from previous produce() calls
    p.poll(0)

    # Asynchronously produce a message, the delivery report callback
    # will be triggered from poll() above, or flush() below, when the message has
    # been successfully delivered or failed permanently.
    p.produce('mytopic', data.encode('utf-8'), callback=delivery_report)

# Wait for any outstanding messages to be delivered and delivery report
# callbacks to be triggered.
p.flush()

Message delivered to mytopic [0]


0